In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.autograd import Variable
import time
import torchvision.models as models
import torchvision.transforms as transforms
import os
from torch.utils.data.sampler import SubsetRandomSampler
import time

In [ ]:
# CIFAR10 Test dataset and dataloader declaration
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
indices = list(range(500))
testSampler = SubsetRandomSampler(indices)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2, sampler=testSampler)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the network
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, 10)

# freeze front layers
ct = 0
for child in model.children():
    ct += 1
    if ct < 5:
        for param in child.parameters():
            param.requires_grad = False
model = model.to(device)
# model.eval()

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [5]:
model.load_state_dict(torch.load('./checkpoints/model.th')['state_dict'])
model.eval()
print(type(model))

<class 'torchvision.models.resnet.ResNet'>


In [6]:
import pickle

def load_cifar10_data(filename):
    with open('data/cifar-10-batches-py/'+ filename, 'rb') as file:
        batch = pickle.load(file, encoding='latin1')

    features = batch['data']
    labels = batch['labels']
    return features, labels

_, trueLabels = load_cifar10_data('test_batch')

In [7]:
import glob
from PIL import Image

for eps in ['0.0', '0.1', '0.2', '0.3', '0.4', '0.5']:
    print(eps)
    startTime = time.time()
    imageTensors = []
    
    images = glob.glob("jsma_perturbed_images/" + eps + "/*png")
    images.sort(key = lambda fileName: int(fileName.split('/')[2].replace('.png', '')))

    for image in images:
        im = Image.open(image)
        imageTensors.append(transforms.ToTensor()(im).unsqueeze_(0))
        im.close()

    correctLabels = 0
    confidence_score_diff = 0.0
    index = 0
    for image in imageTensors:
        out = model(image)
        prob = torch.nn.functional.softmax(out, dim=1)[0]
        _, indices = torch.sort(out, descending=True)
        top2 = [(classes[idx], prob[idx].item()) for idx in indices[0][:2]]
        predictionLabel = top2[0][0]
        trueLabel = classes[trueLabels[index]]
        if predictionLabel == trueLabel:
            correctLabels += 1
            firstScore = top2[0][1]
            secondScore = top2[1][1]
            diff = firstScore - secondScore
            confidence_score_diff += diff
        index += 1
    endTime = time.time()

    print(correctLabels)
    print(confidence_score_diff / (correctLabels * 1.0))
    print(endTime - startTime)


for eps in ['0.0', '0.1', '0.2', '0.3', '0.4', '0.5']:
    startTime = time.time()
    imageTensors = []
    
    images = glob.glob("cw_perturbed_images/" + eps + "/*png")
    images.sort(key = lambda fileName: int(fileName.split('/')[2].replace('.png', '')))

    for image in images:
        im = Image.open(image)
        imageTensors.append(transforms.ToTensor()(im).unsqueeze_(0))
        im.close()

    correctLabels = 0
    confidence_score_diff = 0.0
    index = 0
    for image in imageTensors:
        out = model(image)
        prob = torch.nn.functional.softmax(out, dim=1)[0]
        _, indices = torch.sort(out, descending=True)
        top2 = [(classes[idx], prob[idx].item()) for idx in indices[0][:2]]
        predictionLabel = top2[0][0]
        trueLabel = classes[trueLabels[index]]
        if predictionLabel == trueLabel:
            correctLabels += 1
            firstScore = top2[0][1]
            secondScore = top2[1][1]
            diff = firstScore - secondScore
            confidence_score_diff += diff
        index += 1
    endTime = time.time()

    print(correctLabels)
    print(confidence_score_diff / (correctLabels * 1.0))
    print(endTime - startTime)

0.0
7934
0.8724737979526339
668.6937658786774
0.1
7775
0.866907459492799
635.1673898696899
0.2
7576
0.8567044793669342
627.3614809513092
0.3
7400
0.8396561720693211
517.2511832714081
0.4
7211
0.8348677493678213
401.06687593460083
0.5
7078
0.8273805809854149
401.64978981018066
7932
0.8728522262089601
403.80603861808777
2934
0.6653070619832224
400.24532890319824
2468
0.6237107655214728
396.01755809783936
4343
0.7238999178837513
394.2788782119751
3910
0.7019075953784164
394.37055015563965
3697
0.6841234157694743
418.82414388656616
